tf的graph使用


In [1]:
import tensorflow as tf

# 关于全局default graph当前default graph和生成新graph

In [30]:
#对比graph，这个常量c所属graph是defaut graph
#g是默认graph
sess = tf.Session()
real_default_graph = tf.get_default_graph()#全局
print('real default graph:',real_default_graph)
g = tf.Graph()#生成一个graph
print('g is :',g)
with g.as_default():#局部，as_default，局部get的default graph就是用g设定的。不等于real
    c = tf.constant(30.0)
    assert c.graph is g
    assert c.graph is tf.get_default_graph()
    print('c belongs to current default graph:\t',c.graph ==tf.get_default_graph())
    print('current default is real default?\t',tf.get_default_graph() == real_default_graph)
    print(g)
g=tf.Graph()
with g.as_default():
    c = tf.constant(5.0)
    print(c.graph)
    assert c.graph is g

real default graph: <tensorflow.python.framework.ops.Graph object at 0x7f7734ed9c88>
g is : <tensorflow.python.framework.ops.Graph object at 0x7f7733eb87b8>
c belongs to current default graph:	 True
current default is real default?	 False


In [35]:
#新的c，首先属于current default graph的，但是current default graph变回real了。
c = tf.constant(4.0)
print('c belongs to current default_graph?\t',c.graph is tf.get_default_graph())
print('current default is real default?\t',tf.get_default_graph() == real_default_graph)
print('c belongs to g?\t\t\t\t',c.graph is g)
print(g)
print(tf.Graph())#这算生成一个
print(tf.Graph())#这算生成一个
print(tf.Graph())#这算生成一个
#help(tf.Graph())

c belongs to current default_graph?	 True
current default is real default?	 True
c belongs to g?				 False


## 新例子：variable、reuse和graph。
get_variable本来就是存在则获取，不存在则创建，所以reuse与否关系很大。

In [54]:
#重名限制，如果不设置reuse就不可以有重名的v，但是换不同的graph的话
g1 = tf.Graph()
with g1.as_default():
    v=tf.get_variable('v',initializer=tf.zeros_initializer()(shape=[2,2]))
    #v=tf.get_variable('v')#：ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:\
    print(id(v))
    print(v.graph)
g2 = tf.Graph()
with g2.as_default():
    v=tf.get_variable('v',initializer=tf.ones_initializer()(shape=[2,2]))
    print(id(v))
    print(v.graph)
#分别从不同的graph获取不同的v。
#前文g1和g2都定义了v，默认不让获取重名，必须设置reuse=True：error：v already exists。
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('',reuse=True):
        print(sess.run(tf.get_variable('v')))
with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse = True):
        print(sess.run(tf.get_variable('v')))

140149947700896
140149966017760
[[ 0.  0.]
 [ 0.  0.]]
[[ 1.  1.]
 [ 1.  1.]]


# 例子：用protobuf形式的配置初始化session
记得重启内核，清除不必要的东西。

In [7]:
config = tf.ConfigProto()#
config.gpu_options.allow_growth = True#按需增加GPU容量
print(config)
sess = tf.Session(config=config)
tf.ConfigProto(log_device_placement = True, allow_soft_placement=True)#如果指定的设备不存在，允许自动分配设备，这里没有GPU，所以用CPU执行了。
a = tf.constant([1.0,2.0],name = 'a')
b = tf.constant([1.0,2.0],name = 'b')

g = tf.Graph()
with g.device('/gpu:3'):
    result = a + b
    #print(g.device)
    print('result:',sess.run(result))

gpu_options {
  allow_growth: true
}

<bound method Graph.device of <tensorflow.python.framework.ops.Graph object at 0x7f9502797908>>
result: [ 2.  4.]


In [ ]:
#之前的一个例子，文件名是graph demo，不知道和graph有什么关系了。
def my_func(pred,tensor):
	t = tf.matmul(tensor,tensor)
	with tf.control_dependencies([pred]):
		return t
